In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
!wget https://raw.githubusercontent.com/kunjee17/mahabharata/master/books/14.txt

--2024-08-19 10:51:50--  https://raw.githubusercontent.com/kunjee17/mahabharata/master/books/14.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 529736 (517K) [text/plain]
Saving to: ‘14.txt.1’

14.txt.1            100%[===================>] 517.32K  --.-KB/s    in 0.006s  

2024-08-19 10:51:50 (84.5 MB/s) - ‘14.txt.1’ saved [529736/529736]



In [3]:
file=open("14.txt",'r')
text=file.read()


In [4]:
text[:1000]

'The Mahabharata\n\nof\n\nKrishna-Dwaipayana Vyasa\n\nBOOK 14\n\nASWAMEDHA PARVA\n\nTranslated into English Prose from the Original Sanskrit Text\n\nby\n\nKisari Mohan Ganguli\n\n[1883-1896]\n\nScanned at sacred-texts.com, January 2004. Proofed by John Bruno Hare.\n\n\n\nTHE MAHABHARATA\n\nASWAMEDHA PARVA\n\nSECTION I\n\n(Aswamedhika Parva)\n\nOM! HAVING BOWED down unto Narayana, and Nara the foremost of male\nbeings, and unto the goddess Saraswati, must the word Jaya be uttered.\n\n"Vaisampayana said, \'After the king Dhritarashtra had offered libations\nof water (unto the manes of Bhisma), the mighty-armed[1] Yudhishthira,\nwith his senses bewildered, placing the former in his front, ascended the\nbanks (of the river), his eyes suffused with tears, and dropt down on the\nbank of the Ganga like an elephant pierced by the hunter. Then incited by\nKrishna, Bhima took him up sinking. "This must not be so," said Krishna,\nthe grinder of hostile hosts. The Pandavas, O king, saw Yudhishthir

In [5]:
chars=tuple(set(text))
int2char=dict(enumerate(chars))
char2int={ch:ii for ii,ch in int2char.items()}
encoded=np.array([char2int[ch] for ch in text])

In [6]:
def one_hot_encode(arr, n_labels):

    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''

    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total

    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))

    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[15  4 41 70 62 31  4 31 59  4]
 [34 31 13 70 48 43 70 32 73 42]
 [10 43  4 41 27 50 31 20 12 31]
 [50 13  8 51 49  2 16 70 10 21]
 [70 21 27 13 70 52 12 70 27  4]
 [65 21 41 12 27 70 63 36 73 42]
 [ 4 21 43  4 58 31 70 27  4 41]
 [12 41 42 70 30 41 31  9 41 12]]

y
 [[ 4 41 70 62 31  4 31 59  4 31]
 [31 13 70 48 43 70 32 73 42  4]
 [43  4 41 27 50 31 20 12 31 13]
 [13  8 51 49  2 16 70 10 21 12]
 [21 27 13 70 52 12 70 27  4 36]
 [21 41 12 27 70 63 36 73 42 12]
 [21 43  4 58 31 70 27  4 41 12]
 [41 42 70 30 41 31  9 41 12 24]]


In [11]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else:
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):

    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)

        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))


    def forward(self, x, hidden):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)

        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(train_on_gpu):
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())

                    val_losses.append(val_loss.item())

                net.train() # reset to train mode after iterationg through validation data

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [14]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(76, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=76, bias=True)
)


In [15]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2578... Val Loss: 3.2352
Epoch: 1/20... Step: 20... Loss: 3.1588... Val Loss: 3.1723
Epoch: 1/20... Step: 30... Loss: 3.1630... Val Loss: 3.1725
Epoch: 2/20... Step: 40... Loss: 3.1694... Val Loss: 3.1609
Epoch: 2/20... Step: 50... Loss: 3.1561... Val Loss: 3.1601
Epoch: 2/20... Step: 60... Loss: 3.1262... Val Loss: 3.1592
Epoch: 2/20... Step: 70... Loss: 3.1206... Val Loss: 3.1481
Epoch: 3/20... Step: 80... Loss: 3.1234... Val Loss: 3.1329
Epoch: 3/20... Step: 90... Loss: 3.0747... Val Loss: 3.0953
Epoch: 3/20... Step: 100... Loss: 2.9977... Val Loss: 3.0348
Epoch: 3/20... Step: 110... Loss: 2.9037... Val Loss: 2.9318
Epoch: 4/20... Step: 120... Loss: 2.9337... Val Loss: 2.8751
Epoch: 4/20... Step: 130... Loss: 2.7334... Val Loss: 2.7582
Epoch: 4/20... Step: 140... Loss: 2.6198... Val Loss: 2.6729
Epoch: 5/20... Step: 150... Loss: 2.5941... Val Loss: 2.6198
Epoch: 5/20... Step: 160... Loss: 2.5278... Val Loss: 2.5950
Epoch: 5/20... Step: 170... Loss:

In [16]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''

        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)

        if(train_on_gpu):
            inputs = inputs.cuda()

        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())

        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [18]:
def sample(net, size, prime='The', top_k=None):

    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()

    net.eval() # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [21]:
print(sample(net, 1000, prime='krishna was saying', top_k=5))


krishna was saying alwanss off the sand of men to be
deing
which alous that are cals that time. Wele the
goods of howeres, and a compenter the suct or born and the posels."[19] And the moss free from that was heaven
the formmors
of the pricesting thou has comperied if to the senses of goed
by mounter. When the ere this as the sins of all selfous of sire. Those foremost for all creatudes, the son, O monarch, as the elemothar
of the
gail and all such and the diverse others and say
the san of Parsuva, and the ear, which then are seased that this faritity of thy enceed by thy own sound bech offlersing
him tood, that there, wos thou ars free for all the sicres of the mention of the mind, and he huver it is the menis of this worlds. The worts, worshind those fells, O paistors, accepting a merince and the hero, with the
mone of all
the sich son of Patara, thou hast
that sensite in that fell for the sine. The sacrufice of
his constution of that are seefices. It it subdunce of arowed be the
men

In [22]:
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [23]:
print(sample(loaded, 2000, top_k=5, prime="karna was"))

karna was the monertal as
the priper of all to be stould be that food which the man along the perpersoul bean armonted and that with a condution and seat and wated
on the sive of the great farss). He thousards with many weath, and his beang that from his sacrifice of mine is and the wind. That the foremost of all sensial to the penalos of the son of Dhananjaya,
and that fear of the great sanres, thas and that are spach, who ares of
men, the
his offering the sacrifice, and the sice, and the good of a shall seaked that is as at a tood evan offerend acts was indees, the sacrifices and the son for that soul of the minds and said, -"O beange and said, 'He weal to
han become been
to the mounce. Hen having thas indurged wat one, and and sons off is nether,
that were offered by the sate of the senfis themefore of all the seases, and thou ald and the
senses and thou art the sicce. It induring the seat of
heart, and with me that fire, what woss wind that was,. He who are condect with the mind is